In [1]:
import pandas as pd
import numpy as np
from nameparser import HumanName
import re


In [2]:
# Input file is the Roster Download from College Website. It is a compilation of the available years.
df = pd.read_csv('CornellM.csv')

In [3]:
# Examine the columns, because there are a few different formats used by most colleges
df.columns

Index(['Field7_Text', 'Field7_Link', 'Field1', 'Field8', 'Field9', 'Field11',
       'Field2_Text', 'Field3_Text'],
      dtype='object')

In [4]:
# Rename columns with meaningful labels
df = df.rename(columns = {'Field7_Text':'Name1','Field7_Link':'RosterLink','Field1':'ClassYear', 
                          'Field8':'Hometown','Field9':'HS','Field11':'RosterYear',
                          'Field2_Text':'Name2', 'Field3_Text':'Name3'})

In [5]:
# Note the 'Name' is in 3 possible columns, because the download of each roster by year 
# has a few different formats. Field4 is a duplicate name column, and will be ignored
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 8 columns):
Name1         227 non-null object
RosterLink    227 non-null object
ClassYear     525 non-null object
Hometown      525 non-null object
HS            517 non-null object
RosterYear    525 non-null object
Name2         227 non-null object
Name3         298 non-null object
dtypes: object(8)
memory usage: 32.9+ KB


In [6]:
# Preview the data
df.head()

,Name1,RosterLink,ClassYear,Hometown,HS,RosterYear,Name2,Name3
0,NaN,NaN,Freshman,"Baltimore, MD.",Northwestern College,1946-47 Men's Tennis Roster,NaN,Robert W. Corrigan
1,NaN,NaN,Sophomore,"Laredo, Tex.",Laredo H.,1946-47 Men's Tennis Roster,NaN,Rod R. de Llano
2,NaN,NaN,Senior,"Dover, Del.",Dover H.,1946-47 Men's Tennis Roster,NaN,Jordan K. Dingle
3,NaN,NaN,Freshman,"Ridgewood, NJ",Ridgewood HS,1946-47 Men's Tennis Roster,NaN,Gordon Gardiner
4,NaN,NaN,Sophomore,"Huntington, N.Y.",Horace Mann HS,1946-47 Men's Tennis Roster,NaN,Edward H. Gilbert


In [7]:
# Combine and rename the 'Name' columns, to get a single column with no missing values
# The Name2 field was redundant so can be ignored
# For CornellM there is no 'Field4'
df = df.assign(**{
    'Name': df['Name1'].fillna(df['Name3'])})
df = df.drop(['Name1', 'Name2', 'Name3'] ,axis=1)

In [8]:
# Preview again after consolidating Names columns
df.head()

,RosterLink,ClassYear,Hometown,HS,RosterYear,Name
0,NaN,Freshman,"Baltimore, MD.",Northwestern College,1946-47 Men's Tennis Roster,Robert W. Corrigan
1,NaN,Sophomore,"Laredo, Tex.",Laredo H.,1946-47 Men's Tennis Roster,Rod R. de Llano
2,NaN,Senior,"Dover, Del.",Dover H.,1946-47 Men's Tennis Roster,Jordan K. Dingle
3,NaN,Freshman,"Ridgewood, NJ",Ridgewood HS,1946-47 Men's Tennis Roster,Gordon Gardiner
4,NaN,Sophomore,"Huntington, N.Y.",Horace Mann HS,1946-47 Men's Tennis Roster,Edward H. Gilbert


In [9]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [10]:
#This name was on different rosters, once without an MI
df['Name'][477] = 'James P. Shoffner'

In [11]:
# Some roster downloads have duplicates because of workarounds for the Octoparse roster looping. 
# Check the row count after dropping duplicates 
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525 entries, 0 to 524
Data columns (total 6 columns):
RosterLink    227 non-null object
ClassYear     525 non-null object
Hometown      525 non-null object
HS            517 non-null object
RosterYear    525 non-null object
Name          525 non-null object
dtypes: object(6)
memory usage: 28.7+ KB


In [12]:
# Reorder and sort the columns
df = df[['Name', 'RosterYear', 'ClassYear', 'Hometown', 'HS', 'RosterLink']]
df.sort_values(by=['Name', 'RosterYear'], inplace=True)
df.head()

,Name,RosterYear,ClassYear,Hometown,HS,RosterLink
269,Abedin Sham,2002-03 Men's Tennis Roster,Sophomore,"Mumbai, India",H.R. College of Commerce,https://cornellbigred.com/sports/mens-tennis/r...
257,Abedin Sham,2003-04 Men's Tennis Roster,Junior,"Mumbai, India",H.R. College of Commerce,https://cornellbigred.com/sports/mens-tennis/r...
60,Al Hill,2016-17 Men's Tennis Roster,Freshman,"Atlanta, Ga.",The Westminster Schools,https://cornellbigred.com/sports/mens-tennis/r...
43,Alafia Ayeni,2017-18 Men's Tennis Roster,Freshman,"San Diego, Calif.",Westview HS,NaN
30,Alafia Ayeni,2018-19 Men's Tennis Roster,Sophomore,"San Diego, Calif.",Westview HS,NaN


In [13]:
# Describe is a quick way to see how many unique values are in each column 
df.describe()

,Name,RosterYear,ClassYear,Hometown,HS,RosterLink
count,525,525,525,525,517,227
unique,228,39,6,187,193,227
top,Josh Raff,1952-53 Men's Tennis Roster,Sophomore,"St. Louis, Mo.",Haverford School,https://cornellbigred.com/sports/mens-tennis/r...
freq,4,18,174,11,8,1


In [14]:
# Preview the groupby() stats, which will be used for aggreating to one row per unique name
df.groupby(['Name']).describe()

RosterYear                                             \
                           count unique                            top freq   
Name                                                                          
Abedin Sham                    2      2    2003-04 Men's Tennis Roster    1   
Al Hill                        1      1    2016-17 Men's Tennis Roster    1   
Alafia Ayeni                   3      3    2017-18 Men's Tennis Roster    1   
Albert O. Trostel              1      1    1952-53 Men's Tennis Roster    1   
Alex Kim                       1      1    2004-05 Men's Tennis Roster    1   
...                          ...    ...                            ...  ...   
William S. Greenawalt          2      2  1953-1954 Men's Tennis Roster    1   
William Swift                  2      2    1964-65 Men's Tennis Roster    1   
William T. Meyer               3      3    1955-56 Men's Tennis Roster    1   
Zach Gallin                    3      3    2001-02 Men's Tennis Roster    1   
Zhongming Chen                 4      4    2012-13 Men's Tennis Roster    1   

                      ClassYear                        Hometown         \
                          count unique        top freq    count unique   
Name                                                                     
Abedin Sham                   2      2  Sophomore    1        2      1   
Al Hill                       1      1   Freshman    1        1      1   
Alafia Ayeni                  3      3   Freshman    1        3      1   
Albert O. Trostel             1      1  Sophomore    1        1      1   
Alex Kim                      1      1   Freshman    1        1      1   
...                         ...    ...        ...  ...      ...    ...   
William S. Greenawalt         2      2  Sophomore    1        2      1   
William Swift                 2      2  Sophomore    1        2      1   
William T. Meyer              3      3  Sophomore    1        3      1   
Zach Gallin                   3      3  Sophomore    1        3      1   
Zhongming Chen                4      4     Senior    1        4      1   

                                                   HS         \
                                       top freq count unique   
Name                                                           
Abedin Sham                  Mumbai, India    2     2      1   
Al Hill                       Atlanta, Ga.    1     1      1   
Alafia Ayeni             San Diego, Calif.    3     3      1   
Albert O. Trostel          Milkauwee, Wis.    1     1      1   
Alex Kim                   Beaverton, Ore.    1     1      1   
...                                    ...  ...   ...    ...   
William S. Greenawalt        Hartsdale, NY    2     2      1   
William Swift             Springfield, Pa.    2     2      1   
William T. Meyer       W. Palm Beach, Fla.    3     3      1   
Zach Gallin                 Setauket, N.Y.    3     3      1   
Zhongming Chen             Oakland, Calif.    4     4      1   

                                                           RosterLink         \
                                                  top freq      count unique   
Name                                                                           
Abedin Sham                  H.R. College of Commerce    2          2      2   
Al Hill                       The Westminster Schools    1          1      1   
Alafia Ayeni                              Westview HS    3          0      0   
Albert O. Trostel                              Exeter    1          0      0   
Alex Kim                                 Southside HS    1          0      0   
...                                               ...  ...        ...    ...   
William S. Greenawalt                    Scarsdale HS    2          0      0   
William Swift                          Springfield HS    2          0      0   
William T. Meyer                        Palm Beach HS    3          0      0   
Zach Gallin          

In [15]:
# Create copy of ClassYear in order to aggregate count and keep original unique aggregation when grouping
df['YearsPlayed'] = df['ClassYear']

In [16]:
# Create 'Year' by 1) converting RosterYear to a list
# 2) Loop through the list to use RegEx split function
# 3) Use list comprehension to extract first column of each split row
# 4) Convert string to integer and add 1 to calculate value from first year before dash in yyyy-yy
rosterlist = df['RosterYear'].tolist()
Year=[]
for i in rosterlist:
    Split=int(re.split(r'\D+', i)[0])+1
    Year.append(Split)
df['Year'] = Year

In [17]:
# Use groupby() with aggregation to consolidate columns and condense to one row per Name
dfgroup = df.groupby('Name').aggregate({'Name':'first', 'Year':'max', 'YearsPlayed':'count',\
'RosterYear':'unique', 'ClassYear':'unique', 'Hometown':'first', 'HS':'first', 'RosterLink':'last'})

In [18]:
# Add a column for College, Criteris, and Gender
dfgroup['College'] = 'Cornell University'
dfgroup['Criteria'] = ' Cornell'
dfgroup['Gender'] = 'M'

In [19]:
dfgroup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228 entries, Abedin Sham to Zhongming Chen
Data columns (total 11 columns):
Name           228 non-null object
Year           228 non-null int64
YearsPlayed    228 non-null int64
RosterYear     228 non-null object
ClassYear      228 non-null object
Hometown       228 non-null object
HS             225 non-null object
RosterLink     81 non-null object
College        228 non-null object
Criteria       228 non-null object
Gender         228 non-null object
dtypes: int64(2), object(9)
memory usage: 21.4+ KB


In [20]:
dfgroup

,Name,Year,YearsPlayed,RosterYear,ClassYear,Hometown,HS,RosterLink,College,Criteria,Gender
Name,,,,,,,,,,,
Abedin Sham,Abedin Sham,2004,2,"[2002-03 Men's Tennis Roster, 2003-04 Men's Te...","[Sophomore, Junior]","Mumbai, India",H.R. College of Commerce,https://cornellbigred.com/sports/mens-tennis/r...,Cornell University,Cornell,M
Al Hill,Al Hill,2017,1,[2016-17 Men's Tennis Roster],[Freshman],"Atlanta, Ga.",The Westminster Schools,https://cornellbigred.com/sports/mens-tennis/r...,Cornell University,Cornell,M
Alafia Ayeni,Alafia Ayeni,2020,3,"[2017-18 Men's Tennis Roster, 2018-19 Men's Te...","[Freshman, Sophomore, Junior]","San Diego, Calif.",Westview HS,NaN,Cornell University,Cornell,M
Albert O. Trostel,Albert O. Trostel,1953,1,[1952-53 Men's Tennis Roster],[Sophomore],"Milkauwee, Wis.",Exeter,NaN,Cornell University,Cornell,M
Alex Kim,Alex Kim,2005,1,[2004-05 Men's Tennis Roster],[Freshman],"Beaverton, Ore.",Southside HS,NaN,Cornell University,Cornell,M
...,...,...,...,...,...,...,...,...,...,...,...
William S. Greenawalt,William S. Greenawalt,1955,2,"[1953-1954 Men's Tennis Roster, 1954-55 Men's ...","[Sophomore, Junior]","Hartsdale, NY",Scarsdale HS,NaN,Cornell University,Cornell,M
William Swift,William Swift,1966,2,"[1964-65 Men's Tennis Roster, 1965-66 Men's Te...","[Sophomore, Junior]","Springfield, Pa.",Springfield HS,NaN,Cornell University,Cornell,M
William T. Meyer,William T. Meyer,1958,3,"[1955-56 Men's Tennis Roster, 1956-57 Men's Te...","[Sophomore, Junior, Senior]","W. Palm Beach, Fla.",Palm Beach HS,NaN,Cornell University,Cornell,M


In [21]:
# Use a for loop with nameparser utility HumanName to parse names into First, Middle, Last
namelist = dfgroup['Name'].tolist()
First=[]
Last=[]
Middle=[]
for name in namelist:
    parsename = HumanName(name)
    First.append(parsename.first)
    Last.append(parsename.last)
    Middle.append(parsename.middle)

In [22]:
# Display the elements of parsename.HumanName for last row. title, suffix, nickname are not applicable 
parsename

<HumanName : [
	title: '' 
	first: 'Zhongming' 
	middle: '' 
	last: 'Chen' 
	suffix: ''
	nickname: ''
]>

In [23]:
# Use RegEx to identify Middle values with a period vs. longer names with no period
# Note RegEx syntax "?<!\w\" means don't split the character e.g. keep the period
MidSplit=[]
for i in Middle:
    MidSplit.append(re.split(r'\.*\W+', i)[0])


In [24]:
MiddleName=[]
for i in MidSplit:
    if len(i) > 1:
        MiddleName.append(" "+i)
    else:
        MiddleName.append("")


In [25]:
dfgroup['First'] = First
dfgroup['MidInit'] = Middle
dfgroup['MidName'] = MiddleName
dfgroup['Last'] = Last
dfgroup['FirstLast'] = dfgroup['First'].map(str)+dfgroup['MidName']+' '+dfgroup['Last'].map(str)

In [26]:
dfgroup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228 entries, Abedin Sham to Zhongming Chen
Data columns (total 16 columns):
Name           228 non-null object
Year           228 non-null int64
YearsPlayed    228 non-null int64
RosterYear     228 non-null object
ClassYear      228 non-null object
Hometown       228 non-null object
HS             225 non-null object
RosterLink     81 non-null object
College        228 non-null object
Criteria       228 non-null object
Gender         228 non-null object
First          228 non-null object
MidInit        228 non-null object
MidName        228 non-null object
Last           228 non-null object
FirstLast      228 non-null object
dtypes: int64(2), object(14)
memory usage: 30.3+ KB


In [27]:
# Write the new format to a csv file
#dfgroup.to_csv('CornellW_group.csv', index=False)

In [28]:
# Unique name count is 229, with Roster year count 39. There is a gap missing 1966-67 through 2000-2001, or 34 missing years. 
# 227 LinkedIn URL values were found, but many are likely to be non-matching. 
# The earliest roster year is 1946-47, and there will be dropff of found URL's at some point
# Collapse to unique names, and create boolean for roster years (39 columns)

In [29]:
dfpdf = pd.read_csv('CornellMpdf.csv')

In [30]:
dfpdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 5 columns):
FirstLast      113 non-null object
Last           113 non-null object
First          113 non-null object
Year           113 non-null int64
YearsPlayed    113 non-null int64
dtypes: int64(2), object(3)
memory usage: 4.5+ KB


In [31]:
dfappend = dfgroup.append(dfpdf)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [32]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [33]:
dfappend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341 entries, Abedin Sham to 112
Data columns (total 16 columns):
ClassYear      228 non-null object
College        228 non-null object
Criteria       228 non-null object
First          341 non-null object
FirstLast      341 non-null object
Gender         228 non-null object
HS             225 non-null object
Hometown       228 non-null object
Last           341 non-null object
MidInit        228 non-null object
MidName        228 non-null object
Name           228 non-null object
RosterLink     81 non-null object
RosterYear     228 non-null object
Year           341 non-null int64
YearsPlayed    341 non-null int64
dtypes: int64(2), object(14)
memory usage: 45.3+ KB


In [34]:
dfappend.reset_index(drop=True)
dfappend.set_index(['FirstLast'], inplace=True, drop=False)

In [35]:
# Update College, Critera, and Gender for appended names
dfappend['College'] = 'Cornell University'
dfappend['Criteria'] = ' Cornell'
dfappend['Gender'] = 'M'

In [36]:
cols_to_order = ['Criteria','FirstLast', 'Year', 'YearsPlayed', 'RosterLink', 'Hometown', 'HS']
new_columns = cols_to_order + (dfappend.columns.drop(cols_to_order).tolist())
dfappend = dfappend[new_columns]

In [37]:
# Write the new format to a csv file
dfappend.to_csv('CornellM_RosterGroup.csv', index=True)

In [38]:
dfappend['FirstLast'].to_csv('NamelistCornellM.csv', index=False, header=True)
dfappend['Criteria'].to_csv('CriterialistCornellM.csv', index=False, header=True)

In [39]:
dfappend.sort_values(by=['Year'], inplace=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
dfappend

,Criteria,FirstLast,Year,YearsPlayed,RosterLink,Hometown,HS,ClassYear,College,First,Gender,Last,MidInit,MidName,Name,RosterYear
FirstLast,,,,,,,,,,,,,,,,
John Smith,Cornell,John Smith,1947,1,NaN,"Binghamton, N.Y.",Central H.,[Senior],Cornell University,John,M,Smith,V.,,John V. Smith,[1946-47 Men's Tennis Roster]
Edward Gilbert,Cornell,Edward Gilbert,1947,1,NaN,"Huntington, N.Y.",Horace Mann HS,[Sophomore],Cornell University,Edward,M,Gilbert,H.,,Edward H. Gilbert,[1946-47 Men's Tennis Roster]
Jordan Dingle,Cornell,Jordan Dingle,1947,1,NaN,"Dover, Del.",Dover H.,[Senior],Cornell University,Jordan,M,Dingle,K.,,Jordan K. Dingle,[1946-47 Men's Tennis Roster]
Robert Rubin,Cornell,Robert Rubin,1947,1,NaN,"Washington, DC.",Batto Poly.,[Junior],Cornell University,Robert,M,Rubin,J.,,Robert J. Rubin,[1946-47 Men's Tennis Roster]
David Potter,Cornell,David Potter,1947,1,NaN,"E. Rochester, NY",E. Rochester,[Junior],Cornell University,David,M,Potter,F.,,David F. Potter,[1946-47 Men's Tennis Roster]
Richard Gnaedinger,Cornell,Richard Gnaedinger,1947,1,NaN,"Oak Park, Ill.",Oak Park H.,[Freshman],Cornell University,Richard,M,Gnaedinger,C.,,Richard C. Gnaedinger,[1946-47 Men's Tennis Roster]
Hollis Young,Cornell,Hollis Young,1948,2,NaN,"Brookville, NY.",Hill School,"[Junior, Senior]",Cornell University,Hollis,M,Young,D.,,Hollis D. Young,"[1946-47 Men's Tennis Roster, 1947-48 Men's Te..."
Paul Lansaw,Cornell,Paul Lansaw,1948,2,NaN,"Middletown, Ohio",Middletown H.,"[Freshman, Sophomore]",Cornell University,Paul,M,Lansaw,,,Paul Lansaw,"[1946-47 Men's Tennis Roster, 1947-48 Men's Te..."
Robert Corrigan,Cornell,Robert Corrigan,1948,2,NaN,"Baltimore, MD.",Northwestern College,"[Freshman, Sophomore]",Cornell University,Robert,M,Corrigan,W.,,Robert W. Corrigan,"[1946-47 Men's Tennis Roster, 1947-48 Men's Te..."


In [40]:
dfappend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341 entries, John Smith to Vladislav Melnic
Data columns (total 16 columns):
Criteria       341 non-null object
FirstLast      341 non-null object
Year           341 non-null int64
YearsPlayed    341 non-null int64
RosterLink     81 non-null object
Hometown       228 non-null object
HS             225 non-null object
ClassYear      228 non-null object
College        341 non-null object
First          341 non-null object
Gender         341 non-null object
Last           341 non-null object
MidInit        228 non-null object
MidName        228 non-null object
Name           228 non-null object
RosterYear     228 non-null object
dtypes: int64(2), object(14)
memory usage: 45.3+ KB
